In [41]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
import os 
import datetime
import pytz

## Pipeline to ingest ameriflux and SEBAL timeseries and merge them 
1. Unzipping all amerfilux datasets to a directory
2. Read all the BASE files extracted 
3. Add station metadata to each ameriflux sites 
4. Extract all the column variations of each variable for every site 
5. Process the ameriflux hourly data 
6. Change timezone of the stations to GMT 
7. Read GEESEBAL data 
8. Merge the GEESEBAL dataset to the correspoding ameriflux site 

In [43]:
def process_amflux_hourly(df,SW_IN,SW_OUT,lat,elev,Tair_C,RH):
    df=df.copy()
    df["Datetime"]=pd.to_datetime(df["TIMESTAMP_START"],format = "%Y%m%d%H%M")
    df["Date"]=df["Datetime"].dt.date
    df["Time"]=df["Datetime"].dt.time
    df["doy"]=df["Datetime"].dt.dayofyear
    df=df[ df["Datetime"].dt.year>=2015]
    #Relace -9999 by na
    df=df.replace(-9999,np.nan)
    #---Calculate albedo
    df["Albedo"]=df[SW_OUT]/df[SW_IN]
    #Calculate Rext
    ###**Rext is a hiccup
    df["dec"]=23.45*3.14*np.cos((2*3.14/365)*(172-df["doy"]))/365
    df["dr"]=1+0.017*np.cos(2*3.14*(186-df["doy"])/365)
    df["tao"]=2*3.14*(df["Datetime"].dt.hour-12)/24
    lat_rad=lat*3.14/180
#     df["delta"]=0.409*np.sin((2*3.14*df["doy"]/365)-1.39)
#     df["omega"]=np.arccos(-np.tan(lat*3.14/180)*np.tan(df["delta"]))
    df["Rext"]=1367*(np.sin(df["dec"])*np.sin(lat_rad)+np.cos(df["dec"])*np.cos(lat_rad)*np.cos(df["tao"]))/df["dr"]**2
#     df["Rso"]=(0.75+2*(10**-5)*df[Elevation])*df["Rext"]
    ## Transmissivity
    df["Tao_sw_am"]=df[SW_IN]/df["Rext"]
    ##---Es 
    df["P_calc"]=101.3*((293-0.0065*elev)/293)**5.26
    df["es"]=0.6108*np.exp((17.27*df[Tair_C])/(df[Tair_C]+237.3))
    df["ea"]=df[RH]*df["es"]/100
    df["W_calc"]=df["ea"]*0.14*df["P_calc"]*df["ea"]+2.1
    df["Rso"]=(0.75+2*(10**-5)*elev)*df["Rext"]
    ## Transmissivity using dingman 
    df["gamma_dust"]=0.1
    df["W_prata"]=df["ea"]*10/(df[Tair_C]+273.16)
    df["zenith"]=(np.sin(df["dec"])*np.sin(lat_rad)+np.cos(df["dec"])*np.cos(lat_rad)*np.cos(df["tao"]))
    df["Mopt"]=1/df["zenith"]
    df["asa"]=-0.124-0.0207*df["W_prata"]
    df["bsa"]=-0.0682-0.0248*df["W_prata"]
    df["Tao_sa"]=np.exp(df["asa"]+df["bsa"]*df["Mopt"])
    df["Tao_ding"]=df["Tao_sa"]-df["gamma_dust"]
    ## transmissivit insitu using the sebal formula 
    df["Tao_sebal_insitu"]=0.35+0.627*np.exp(((-0.00146*df["P_calc"])/(1*df["zenith"]))-0.075*(df["W_calc"]/df["zenith"])**0.4)
    return df


In [44]:
##Unzip the ameriflux files
# os.chdir("C:\\Rouhin_Lenovo\\US_project\\Untitled Folder\\ET_ameriflux_met")
# file_list=os.listdir()
# import zipfile
# target_file="C:\\Rouhin_Lenovo\\US_project\\Untitled Folder\\Unzipped"
# for i in range(len(file_list)):
#     with zipfile.ZipFile(file_list[i], 'r') as zip_ref:
#         zip_ref.extractall(target_file)

In [45]:
##First is to read only relevant "BASE" files 
new_dir="C:\\Rouhin_Lenovo\\US_project\\Untitled Folder\\Unzipped\\"
os.chdir(new_dir)
new_files=os.listdir()
new_files
file_names=[]
af_data=[]
for i in range(len(new_files)):
    if "BASE" in new_files[i]:
#         print()
        file_names.append(new_files[i].split("_")[1])
        af_data.append(pd.read_csv(new_files[i],skiprows=2))
        

C:\Users\rouhi\AppData\Local\Temp\ipykernel_40324\2545396564.py:12: DtypeWarning: Columns (29,38) have mixed types. Specify dtype option on import or set low_memory=False.
  af_data.append(pd.read_csv(new_files[i],skiprows=2))
C:\Users\rouhi\AppData\Local\Temp\ipykernel_40324\2545396564.py:12: DtypeWarning: Columns (38) have mixed types. Specify dtype option on import or set low_memory=False.
  af_data.append(pd.read_csv(new_files[i],skiprows=2))


In [46]:
## Only read files that are on the station list 
st_am=pd.read_csv("C:\\Rouhin_Lenovo\\US_project\\Untitled Folder\\Station_List\\AMERIFLUX_SWC_Met_US.csv",encoding="unicode_escape")

In [47]:
all_st=[]
for i in range(len(af_data)):
    for j in range(st_am.shape[0]):
        if file_names[i]==st_am["Name"].iloc[j]:
            af_data[i]["Name"]=st_am["Name"].iloc[j]
            af_data[i]["Long"]=st_am["Long"].iloc[j]
            af_data[i]["Lat"]=st_am["Lat"].iloc[j]
            af_data[i]["Elevation"]=st_am["Elev (m)"].iloc[j]
            af_data[i]["State"]=st_am["State"].iloc[j]
            af_data[i]["Timezone"]=st_am["Timezone"].iloc[j]
#             af_data[i]["AmeriFlux BASE Data"]=st_am["AmeriFlux BASE Data"].iloc[j]
            af_data[i]["Veg"]=st_am["Veg"].iloc[j]
            af_data[i]["Clim"]=st_am["Clim"].iloc[j]
            af_data[i]["MAT (°C)"]=st_am["MAT (°C)"].iloc[j]
            af_data[i]["MAP (mm)"]=st_am["MAP (mm)"].iloc[j]            
            all_st.append(af_data[i])
del af_data

### Merge all the datasets with GEESEBAL datasets 
1. Extract all column variations for the function calls 
2. Establish timezone for each site for the merging
This will require a column of timezone. 
    - How to add the timezones
        - One way is to do it manually 
        - or could use google/microsft api

In [48]:
## Now the column names are not the same so we extract all cthe variations of the columns names 
## Then we basically take the most popular column name as first choice and if that doesn't work 
# we take the first element of each list. 
sw_in_list=[]
sw_out_list=[]
TA_list=[]
RH_list=[]
LW_IN_list=[]
LW_OUT_list=[]
NETRAD_list=[]
# le=[]
# h=[]
# g=[]
for i in range(len(all_st)):
    ##
    sw_in_list.append(sorted([col for col in all_st[i].columns if "SW_IN" in col],key=len))
    sw_out_list.append(sorted([col for col in all_st[i].columns if "SW_OUT" in col],key=len))
    TA_list.append(sorted([col for col in all_st[i].columns if col.startswith("TA_")],key=len))
    RH_list.append(sorted([col for col in all_st[i].columns if "RH" in col],key=len))
    LW_IN_list.append(sorted([col for col in all_st[i].columns if "LW_IN" in col],key=len))
    LW_OUT_list.append(sorted([col for col in all_st[i].columns if "LW_OUT" in col],key=len))
    NETRAD_list.append(sorted([col for col in all_st[i].columns if "NETRAD" in col],key=len))
#     le.append(sorted([col for col in all_st[i].columns if "LE" in col],key=len))
    



## Selections 
-SW_IN- we call SW_IN_1_1_1

-SW_OUT: SW_OUT 

-TA: TA 

-RH; RH 

In [49]:
## calling 
# Processed All_st
all_st_p1=[]
for i in range(len(all_st)):
    if "TA" in all_st[i].columns:
        all_st_p1.append(process_amflux_hourly(all_st[i],sw_in_list[i][0],sw_out_list[i][0],all_st[i]["Lat"].iloc[0],all_st[i]["Long"].iloc[0]\
              ,"TA",RH_list[i][0]))
    else:
        all_st_p1.append(process_amflux_hourly(all_st[i],sw_in_list[i][0],sw_out_list[i][0],all_st[i]["Lat"].iloc[0],all_st[i]["Long"].iloc[0]\
  ,TA_list[i][0],RH_list[i][0]))     

In [50]:
del all_st

In [52]:
# Write the processed hourly files
op_dir="C:\\Rouhin_Lenovo\\US_project\\GEE_SEBAL_Project\\Csv_Files\\Processed_AF_SEBAL\\"
for i in range(len(all_st_p1)):
    all_st_p1[i].to_csv(op_dir+all_st_p1[i]["Name"].iloc[0]+".csv")

In [19]:
# all_st_p1=[]
# import os 
# os.chdir("C:\\Rouhin_Lenovo\\US_project\\GEE_SEBAL_Project\\Csv_Files\\Processed_AF_SEBAL\\")
# file_list=os.listdir()
# for i in range(len(file_list)):
#     all_st_p1.append(pd.read_csv(file_list[i],parse_dates=["Datetime"]))
# file_list

In [53]:
all_st_p1[0]["Datetime"].dt.minute
# pd.to_datetime(all_st_p1[0]["TIMESTAMP_START"],format = "%Y%m%d%H%M")

210384     0
210385    30
210386     0
210387    30
210388     0
          ..
324079    30
324080     0
324081    30
324082     0
324083    30
Name: Datetime, Length: 113700, dtype: int64

In [54]:
## Merging 
### Read GEESEBAL Files 
os.chdir("C:\\Rouhin_Lenovo\\US_project\\GEE_SEBAL_Project\\Csv_Files\\Test_files\\All_netrad\\AFLUX_SEBAL_Full")
file_list=os.listdir()
sebal=[]
sebal_names=[]
for i in range(len(file_list)):
    sebal_names.append(file_list[i].split(".")[0])
    sebal.append(pd.read_csv(file_list[i],parse_dates=["date"]))

In [55]:
def convert_timezone_gmt(df,state):
    if state=="WA" or state=="OR" or state=="NV"or state=="CA":
        tz = pytz.timezone('US/Pacific')
    #Still more states to add
    elif state=="AZ" or state=="WY" or state=="UT" or state=="CO" or state=="NM" or state=="ID": 
        tz = pytz.timezone('US/Mountain')
    elif state=="OK" or state=="AR" or state=="IL" or state=="ND" or state=="WI" or state=="AL" \
    or state=="KS" or state=="MO" or state=="TX" or state=="NE" or state=="MN":
        tz=pytz.timezone('US/Central')
    else:
        tz=pytz.timezone('US/Eastern')
    df["Datetime_Local"]=df["Datetime"].dt.tz_localize(tz, nonexistent='shift_forward',ambiguous=False)
    df["Datetime_GMT"]=df["Datetime_Local"].dt.tz_convert('GMT')
    return df
## Change local timezones to GMT for merging 
all_st_p2=[]
for i in range(len(all_st_p1)):
    all_st_p2.append(convert_timezone_gmt(all_st_p1[i],all_st_p1[i]["State"].iloc[0]))

In [56]:
sebal[0]["date"]

0     2015-01-04 18:21:48.016999936
1     2015-01-11 18:27:35.119000064
2     2015-01-27 18:27:29.553999872
3     2015-02-05 18:21:16.744999936
4     2015-02-21 18:21:10.128000000
                   ...             
481   2022-12-09 18:21:53.683000064
482   2022-12-25 18:21:45.176999936
483   2022-12-09 18:22:17.573999872
484   2022-12-25 18:22:09.063000064
485   2022-12-16 18:28:00.224999936
Name: date, Length: 486, dtype: datetime64[ns]

In [57]:
## Creat a time variable fr
def get_coinciding_amflux_data(df_am,df_ls):
    df_ls["Datetime_Local"]=df_ls["date"].dt.tz_localize("GMT", nonexistent='shift_forward',ambiguous=False)
    df_ls["Datetime_GMT"]=df_ls["Datetime_Local"].dt.round("H")
    df_merge=pd.merge(df_ls,df_am,on="Datetime_GMT",how="left")
    return df_merge
# var_merged=get_coinciding_amflux_data(var_new_tz,var_ls)
# srg_merge=get_coinciding_amflux_data(srg_new_tz,srg_ls)
# var_merged["Date"]=pd.to_datetime(var_merged["Date"])
# srg_merge["Date"]=pd.to_datetime(srg_merge["Date"])

In [58]:
sebal[0]

,Unnamed: 0,id,longitude,latitude,date,UB,B,GR,R,NIR,...,AirT_G,RH_G,ux_G,SW_Down,cold_pixel_lat,cold_pixel_lon,cold_pixel_ndvi,cold_pixel_sum,cold_pixel_temp,zenith_angle
0,0,LC08_040035_20150104,-116.693177,36.765395,2015-01-04 18:21:48.016999936,NaN,NaN,NaN,NaN,NaN,...,7.691125,32.189507,0.519157,98.010893,35.440987,-115.458623,0.204961,99326,276.529744,62.355572
1,1,LC08_041034_20150111,-116.693177,36.765395,2015-01-11 18:27:35.119000064,NaN,NaN,NaN,NaN,NaN,...,8.479344,65.328119,0.591262,81.198365,37.545191,-118.077111,0.529581,152678,276.054639,62.934298
2,2,LC08_041034_20150127,-116.693177,36.765395,2015-01-27 18:27:29.553999872,NaN,NaN,NaN,NaN,NaN,...,10.706054,72.500746,2.275621,106.739166,37.294105,-116.451894,0.377359,48355,275.401798,60.195602
3,0,LC08_040034_20150205,-116.693177,36.765395,2015-02-05 18:21:16.744999936,NaN,NaN,NaN,NaN,NaN,...,14.636895,37.250555,1.084385,171.346228,38.207235,-115.697795,0.359773,297666,278.857416,57.955807
4,1,LC08_040034_20150221,-116.693177,36.765395,2015-02-21 18:21:10.128000000,0.125545,0.142788,0.189840,0.235957,0.275145,...,15.146092,12.496244,0.572277,207.888058,38.152855,-115.623437,0.325179,520234,281.663610,52.988954
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
481,0,LC08_040034_20221209,-116.693177,36.765395,2022-12-09 18:21:53.683000064,0.114352,0.134455,0.179995,0.225672,0.267005,...,6.317622,37.865336,0.187745,131.064454,38.112599,-115.558150,0.700816,435516,271.453985,62.823532
482,1,LC08_040034_20221225,-116.693177,36.765395,2022-12-25 18:21:45.176999936,0.115837,0.137040,0.182443,0.226662,0.266483,...,12.505456,36.430663,0.957792,125.960258,37.908594,-115.420184,0.578358,428545,276.697227,63.849188
483,2,LC08_040035_20221209,-116.693177,36.765395,2022-12-09 18:22:17.573999872,0.122217,0.138745,0.178868,0.223775,0.265547,...,6.323166,37.843925,0.189139,131.064454,36.103749,-117.094879,0.266055,488752,275.357363,61.560946
484,3,LC08_040035_20221225,-116.693177,36.765395,2022-12-25 18:22:09.063000064,0.125848,0.142925,0.181287,0.224243,0.264612,...,12.515694,36.395801,0.955136,125.960258,36.545079,-115.270421,0.502215,465335,280.785179,62.609351


In [3]:
merged=[]
for i in range(len(sebal)):
    for j in range(len(all_st_p2)):
        if sebal_names[i]==all_st_p2[j]["Name"].iloc[0]:
            print(all_st_p2[j]["Name"].iloc[0])
            merged.append(get_coinciding_amflux_data(all_st_p2[j],sebal[i]))

NameError: name 'sebal' is not defined

In [60]:
## Save these bad boys
op_dir="C:\\Rouhin_Lenovo\\US_project\\GEE_SEBAL_Project\\Csv_Files\\Merged_files\\"
for i in range(len(merged)):
    merged[i].to_csv(op_dir+merged[i][merged[i]["Name"].notna()]["Name"].iloc[0]+".csv")

In [2]:
merged[0].columns.to_list()

NameError: name 'merged' is not defined